# I. DATASET GENERATING

In [1]:
import cv2
import os

In [2]:
def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            cropped_face = img[y:y+h, x:x+w]
            return cropped_face

    # Chemin vers le dossier principal sur le bureau
    project_path = os.path.join(os.path.expanduser("~"), "Desktop", "BMproject-cmpmnt")
    # Chemin vers le sous-dossier 'captured_img'
    captured_img_path = os.path.join(project_path, "captured_img")

    # Vérifier si le dossier existe, sinon le créer
    #if not os.path.exists(captured_img_path):
        #os.makedirs(captured_img_path)

    cap = cv2.VideoCapture(0)
    id = 1
    img_id = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        if face_cropped(frame) is not None:
            img_id += 1
            face = cv2.resize(face_cropped(frame), (200, 200)) # Ici c'est la dimension de chaque image capturée 200 x 200
            #face_gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY) Ceci traite les images en dimension de gri qui le fait sortir en noire et blanc
            file_name_path = os.path.join(captured_img_path, f"user.{id}.{img_id}.jpg")
            
            #cv2.imwrite(file_name_path, face_gray) 
            cv2.imwrite(file_name_path, face) # Donc ici nous appellons directement face sans passer par face_gray pourque 
            #les images soient comme telles càd en couleur 
            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("cropped face", face)

            if cv2.waitKey(1) == ord('q') or img_id >= 100:
                break

    cap.release()
    cv2.destroyAllWindows()
    print("Collection complete!")

# Appeler la fonction pour démarrer la collecte de données
#generate_dataset()

# II. CLASSIFIER TRAINNING AND SAVING

In [3]:
from PIL import Image
import os
import numpy as np
import cv2

In [4]:
def train_classifier(data_dir):
    path = [os.path.join(data_dir,f) for f in os.listdir(data_dir)]
    faces = []
    ids = []
    
    for image in path:
        img = Image.open(image).convert('L'); # Pour convertir les images en niveaux de gris
        imageNp = np.array(img, 'uint8') # Elle convertit l'image "img", qui est un objet de type PIL.Image, en un tableau NumPy (numpy.ndarray).
        # Le second argument 'uint8' spécifie que les éléments du tableau seront de type entier non signé sur 8 bits. 
        # Cela signifie que chaque valeur dans le tableau sera comprise entre 0 et 255, ce qui est approprié pour 
        # représenter les niveaux d'intensité des pixels dans une image en niveaux de gris.
        id = int(os.path.split(image)[1].split(".")[1])# Pour converit les id en entier

        # NB: Cette conversion est essentielle pour le traitement d'images avec OpenCV, car OpenCV fonctionne principalement avec des tableaux NumPy. 
        # En utilisant un tableau de type uint8, 
        # vous pouvez ensuite appliquer des opérations de traitement d'image, 
        # comme la détection de visages ou d'autres transformations.
        
        faces.append(imageNp) # Ajoute le tableau NumPy "imageNp", qui représente une image de visage en niveaux de gris, à la liste faces.
        ids.append(id) # Ajoute l'identifiant "id" (qui représente l'identité de la personne associée à l'image) à la liste ids.
    ids = np.array(ids) # Convertir la liste en tableau numpy



    # NB: (1) Cette liste ids accumule tous les identifiants correspondants aux images de visages, 
          #permettant de relier chaque image à une personne spécifique lors de l'entraînement.
    # NB: (2) Ces deux listes (faces et ids) sont essentielles pour entraîner le modèle de reconnaissance faciale. 
             # Lors de l'entraînement, le modèle apprend à associer chaque image de visage à son identifiant correspondant, 
            # ce qui lui permet de reconnaître des visages dans des images futures
                  
    # ====== TRAINING THE CLASSIFIER AND SAVE
                   
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")
train_classifier("captured_img")

    

# III. FACE DETECTING AND NAMING